In [1]:
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
%matplotlib inline

In [2]:
# Site to use: https://www.tdcj.state.tx.us/death_row/dr_executed_offenders.html

In [3]:
# Offender site syntax: https://www.tdcj.state.tx.us/death_row/dr_info/firstlastname'.html

In [4]:
# GET ALL THE OFFENSE SITES TO SCRAP
#define site to use
url = 'https://www.tdcj.state.tx.us/death_row/dr_executed_offenders.html'
#get content of main page
resp = requests.get(url)
#put this into beautiful soup format
soup = BeautifulSoup(resp.text)
#get just the content of the spells
info = soup.table
#pull all the links from the spell table
offenses = info.find_all('a')
#make a list of all the unique links from spells (they're listed twice because of 'open in new window')
offense_links = []
for item in offenses:
    link = item.get('href')
    if link not in offense_links:
        offense_links.append('https://www.tdcj.state.tx.us/death_row/'+link)


/Users/mgnbecker/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [5]:
# CREATE LISTS OF RELEVANT FEATURES
name = []
dob = []
age_time_of_offense = []
education_level = []
date_of_offense = []
race = []
gender = []
native_state = []
prior_occupation = []
prior_prison_record = []
summary_of_incident = []

In [6]:
attributes = [name,dob,age_time_of_offense,education_level,date_of_offense,race,gender,native_state,prior_occupation,prior_prison_record,summary_of_incident]

In [18]:
import requests, re
from bs4 import BeautifulSoup
import pandas as pd

def read_table(url):
   soup = BeautifulSoup(requests.get(url).content)
   return soup.find(lambda tag:tag.name == 'table')

def parse_table(url):
   table = read_table(url)
   return [[td.get_text(strip=True)
            if td.find("a") is None 
            else url + td.find("a")["href"] 
            for td in tr.find_all("td") if td.get_text(strip=True) is not u""] 
            for tr in table.find_all("tr")]

def parse_incident(url):
    soup = BeautifulSoup(requests.get(url).content)
    incidents = [incidents.text.strip() for statement in soup.findAll('p')]
    try:
        start = incidents.index(u'Summary of Incident:')
    except:
        start = 0
    incidents = [incidents[i] for i in range(0, len(incidents)) if i > start]
    return u''.join(incidents)

columns = ['execution', 'offender_information', 'last_statement', 'last_name',
           'first_name', 'tdcj_number', 'age', 'date', 'race', 'county', 'summary_of_incident']
data = parse_table('http://www.tdcj.state.tx.us/death_row/dr_executed_offenders.html')
del data[0]
data = pd.DataFrame(data, columns = columns)
data.ix[:, (1,2)] = data.ix[:, (1,2)].applymap(lambda x: re.sub('dr_executed_offenders.html', '', x))
data['incident_text'] = data['incidents'].map(lambda x: parse_incident(x))
data.to_csv('incidents_data.csv', index=False, encoding='utf-8')

AssertionError: 11 columns passed, passed data had 10 columns

In [10]:
#Need help catering this code to my webpage


# ITERATE THROUGH THE PAGES GETTING INFORMATION
for link in offense_links:
    print link
    try:
        content = requests.get(link)
        offense_info =  BeautifulSoup(content.text)

        head = offense_info.find_all('h1')
        for item in head:
            title.append(item.find('span').find(text=True))

        body = offense_info.findAll('p')

        #school.append(body[1].find(text=True))

        #fine_details = body[2].find_all('strong')
        #level.append(fine_details[0].find(text=True))
        #casting_time.append(fine_details[1].find(text=True))
        #spell_range.append(fine_details[2].find(text=True))
        #components.append(fine_details[3].find(text=True).split(', '))
        #duration.append(fine_details[4].find(text=True))

        #description.append(body[3])

        #if 'Page' in body[4].find(text=True):
            # source.append(re.findall(r'[^\n\s].*[^(\s{3,})]', body[4].find(text=True))[0])
            #sourcenum = 4
            #classesnum = 5
            #higher_class.append('n/a')
        #else:
         #   sourcenum = 5
            #classesnum = 6
            #higher_class.append(body[4])

        #source.append(re.findall(r'[^\n\s].*[^(\s{3,})]', body[sourcenum].find(text=True))[0])

        #messyclasses = body[classesnum].find_all('a')
        #classes = []
        #for item in messyclasses:
         #   classes.append(re.findall(r'[^/]*?$', item.get('href'))[0])
        #spell_classes.append(classes)
    #except:
        print 'error'

IndentationError: unexpected unindent (<ipython-input-10-78fcf4e29f65>, line 46)

In [ ]:
#Haven't run this cell yet.

#CREATE DATAFRAME

summary_of_incident = []
index = ['name','dob','age_time_of_offense','education_level','date_of_offense','race','gender','native_state','prior_occupation','prior_prison_record','summary_of_incident']
offense_df = pd.DataFrame(attributes, index=index)

offense_df

offense_df.rename(index={'offense_range':'range'}, inplace=True)

offense_df.to_csv('offense.csv', encoding='utf-8')

offenseforjson = pd.read_csv('spells.csv')
offenseforjson.set_index('Unnamed: 0', inplace=True)
offenseforjson.columns = offenseforjson.loc['title']
offenseforjson.drop('title',axis=0,inplace=True)

offenseforjson
offenseforjson.to_json('offense.json')
